In [55]:
# import os
# print("PATH: %s\n" % os.environ['PATH'])
# print("SPARK_HOME: %s" % os.environ['SPARK_HOME'])
# print("PYSPARK_PYTHON: %s" % os.environ['PYSPARK_PYTHON'])
# print("PYSPARK_DRIVER_PYTHON: %s" % os.environ['PYSPARK_DRIVER_PYTHON'])

In [56]:
# pip install py4j findspark

In [57]:
# import findspark
# findspark.init()

In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, initcap  # Some functions that later will be useful

spark = SparkSession \
        .builder \
        .appName("Reading CSV") \
        .getOrCreate()

In [59]:
df = spark.read.load("C:/Users/nktun/Downloads/supermarket_sales - Sheet1.csv",
                    format='csv',
                    sep = ',',
                    header='true',
                    inferSchema='true').cache()

In [60]:
df.show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|     Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715| 1/5/2019|2023-06-06 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22| 3/8/2019|2023-06-06 10:29:00|       Cash|  76.4|            4.761904762|      

In [61]:
# Also has groupBy like in pandas
gender = df.groupBy('Gender').count()
gender.show()

+------+-----+
|Gender|count|
+------+-----+
|Female|  501|
|  Male|  499|
+------+-----+



In [62]:
df.columns

['Invoice ID',
 'Branch',
 'City',
 'Customer type',
 'Gender',
 'Product line',
 'Unit price',
 'Quantity',
 'Tax 5%',
 'Total',
 'Date',
 'Time',
 'Payment',
 'cogs',
 'gross margin percentage',
 'gross income',
 'Rating']

In [63]:
# Creating a temp table is necessary to perform queries using SQL
df.createOrReplaceTempView("SM_Sales")
results = spark.sql('SELECT * from SM_Sales WHERE `Unit Price` < 15 AND Quantity < 10 AND Gender = "Female" AND `Customer type` = "Normal"')
results.show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity|Tax 5%|   Total|     Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|871-39-9221|     C|Naypyitaw|       Normal|Female|Electronic access...|     12.45|       6| 3.735|  78.435| 2/9/2019|2023-06-06 13:11:00|       Cash|  74.7|            4.761904762|       3.735|   4.1|
|586-25-0848|     A|   Yangon|       Normal|Female|   Sports and travel|     12.34|       7| 4.319|  90.699| 3/4/2019|2023-06-06 11:19:00|Credit card| 86.38|            4.761904762|       4.31

In [64]:
output = spark.sql('SELECT COUNT(*) as total, City from SM_Sales GROUP BY City')
output.show()

+-----+---------+
|total|     City|
+-----+---------+
|  328|Naypyitaw|
|  332| Mandalay|
|  340|   Yangon|
+-----+---------+



In [65]:
abc = spark.sql('SELECT Branch, SUM(Total) as SUM FROM SM_Sales GROUP BY Branch ORDER BY Branch ASC')
abc.show()

+------+------------------+
|Branch|               SUM|
+------+------------------+
|     A| 106200.3705000001|
|     B|106197.67199999996|
|     C|110568.70649999994|
+------+------------------+



In [71]:
abc.write.format('json').save('D:/abc2.json')
# abc.coalesce(1).write.option("header","true").format("csv").save("abc1.csv")

Py4JJavaError: An error occurred while calling o254.save.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileO